In [164]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

In [186]:
df = pd.read_csv("cleaned_verses_only.csv", sep="~")

In [187]:
print(len(df))
df.head()

16036


,Unnamed: 0,rapper,song,year,lyrics
0,0,Cordae,RNP,2019.0,One of y'all niggas gon' be fuckin' with me|I ...
1,1,Cordae,Kung Fu,2018.0,Day-Day-Daytrip took it to 10|Let me tell you ...
2,2,Cordae,Bad Idea,2019.0,Ooh|Ooh|Yeah uh|Bongo ByTheWay|I know myself a...
3,3,Cordae,Have Mercy,2019.0,Kid|Ugh|Sweet Lord please have mercy|Baby Jesu...
4,4,Cordae,Old Niggas,2018.0,Damn Mizzy Mauri got beats|Uh old niggas and n...


In [188]:
df["lyrics"] = df["lyrics"].apply(lambda text: str(text).replace('|', ' \n'))

In [189]:
df["lyrics_splitted"] = df["lyrics"].apply(lambda text: str(text).replace('\n', ' '))

In [190]:
df["lyrics_splitted_lines"] = df["lyrics"].apply(lambda text: str(text).splitlines())

In [191]:
df['lyrics_tokenized'] = df["lyrics_splitted"].apply(lambda text: ' '.join(nltk.word_tokenize(text)))

In [192]:
lemmatizer = WordNetLemmatizer()
df["lyrics_lemmatized"] = df["lyrics_tokenized"].apply(lambda text: [lemmatizer.lemmatize(word) for word in text.split(' ')])

In [193]:
df["formula_num_sent"] = df["lyrics_splitted_lines"].apply(lambda text: len(text))

In [194]:
df_slurs = pd.read_csv("slurs.csv", sep=";")

In [195]:
df_slurs.head()

,text,canonical_form_1,canonical_form_2,canonical_form_3,category_1,category_2,category_3,severity_rating,severity_description
0,69,69,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
1,@55,ass,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
2,rat baztad,bastard,NaN,NaN,animal references,NaN,NaN,1.8,Strong
3,ape shit,ape,shit,NaN,bodily fluids / excrement,NaN,NaN,1.6,Strong
4,ape shite,ape,shit,NaN,bodily fluids / excrement,animal references,NaN,1.6,Strong


In [196]:
slurs = dict(zip(df_slurs.text, df_slurs.severity_rating))

In [197]:
def get_matched_words(df, slurs: dict) -> list:
    df["slurs"] = np.nan
    df["num_slurs"] = np.nan
    df["formula_song"] = np.nan
    for ind, (_, row) in tqdm(enumerate(df.iterrows())):
        match_words = []
        new_set = set(row["lyrics_lemmatized"])
        for word in new_set:
            if word in slurs:
                match_words.append(word)
        row["slurs"] = match_words
        row["num_slurs"] = len(row["slurs"])
        
        formula_song = 0.0
        for word in match_words:
            formula_song += float(slurs[word])
        row["formula_song"] = formula_song / len(row["lyrics_tokenized"])
        
        df.at[ind] = row

In [198]:
get_matched_words(df, slurs)

16036it [00:19, 803.96it/s]


In [199]:
df.head()

,Unnamed: 0,rapper,song,year,lyrics,lyrics_splitted,lyrics_splitted_lines,lyrics_tokenized,lyrics_lemmatized,formula_num_sent,slurs,num_slurs,formula_song
0,0,Cordae,RNP,2019.0,One of y'all niggas gon' be fuckin' with me \n...,One of y'all niggas gon' be fuckin' with me I...,"[One of y'all niggas gon' be fuckin' with me ,...",One of y'all niggas gon ' be fuckin ' with me ...,"[One, of, y'all, nigga, gon, ', be, fuckin, ',...",57,"[motherfuckin, fucking, bitch, nigga, Fuck, fu...",6.0,0.004926
1,1,Cordae,Kung Fu,2018.0,Day-Day-Daytrip took it to 10 \nLet me tell yo...,Day-Day-Daytrip took it to 10 Let me tell you...,"[Day-Day-Daytrip took it to 10 , Let me tell y...",Day-Day-Daytrip took it to 10 Let me tell you ...,"[Day-Day-Daytrip, took, it, to, 10, Let, me, t...",88,"[fucked, fucking, goddamn, bitch, nigga, shit]",6.0,0.003260
2,2,Cordae,Bad Idea,2019.0,Ooh \nOoh \nYeah uh \nBongo ByTheWay \nI know ...,Ooh Ooh Yeah uh Bongo ByTheWay I know myse...,"[Ooh , Ooh , Yeah uh , Bongo ByTheWay , I know...",Ooh Ooh Yeah uh Bongo ByTheWay I know myself a...,"[Ooh, Ooh, Yeah, uh, Bongo, ByTheWay, I, know,...",73,"[nigga, shit]",2.0,0.001238
3,3,Cordae,Have Mercy,2019.0,Kid \nUgh \nSweet Lord please have mercy \nBab...,Kid Ugh Sweet Lord please have mercy Baby J...,"[Kid , Ugh , Sweet Lord please have mercy , Ba...",Kid Ugh Sweet Lord please have mercy Baby Jesu...,"[Kid, Ugh, Sweet, Lord, please, have, mercy, B...",86,"[bitch, nigga, shit, Fuck]",4.0,0.002625
4,4,Cordae,Old Niggas,2018.0,Damn Mizzy Mauri got beats \nUh old niggas and...,Damn Mizzy Mauri got beats Uh old niggas and ...,"[Damn Mizzy Mauri got beats , Uh old niggas an...",Damn Mizzy Mauri got beats Uh old niggas and n...,"[Damn, Mizzy, Mauri, got, beat, Uh, old, nigga...",64,"[nigga, bullshit, fuckin, suck]",4.0,0.002336


In [201]:
df["lyrics"][0]

"One of y'all niggas gon' be fuckin' with me \nI got two bad bitches gon' be rubbin' my feet \nI got three young niggas down to bust out the heat \nAnd all y'all ain't got nothin' on me \nOoh not one of y'all niggas gon' be fuckin' with me \nI got two bad bitches gon' be rubbin' my feet \nI got three young niggas down to bust out the heat \nAnd all y'all ain't got nothin' on me \nUh okay put your fucking hands up this the fucking anthem \nSmiling 'cause I'm young rich black and I'm handsome \nNot to mention wealthy ass on her healthy \nYoung millionaire what the fuck can you tell me? Smell me? \nNigga that's Chanel cologne \nI'm in Europe with the tourists with no cellular phone \nLike ooh sound like rich nigga problems \nI hit a bad bitch with a fistful of condoms \nAnd the randomness of risky ménages \nLike get the head right she can get what she wanted \nThe spits then flaunt it my drip like a faucet \nShe told me she was prego I ain't even take the motherfuckin' dick out my pocket 